In [2]:
from datetime import datetime

import pandas as pd
import numpy as np

import codecs, json 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.metrics import mean_squared_error

import scipy

In [3]:
with open('/Users/Navid/Documents/MPENM/Project Course /Python/data_rich_ac', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    print(json_data)    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
# json_data is a disc 
# The tables is a list and header is dict
# The tables list contains dict -> This dict contains the data
#json_data['tables'][1]['table']

In [143]:
df = pd.DataFrame([])

for j in range(len(json_data['tables'])):
    temp_df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,0:7],
                           columns = json_data['tables'][j]['header']['variables'][0:7])
    temp_df['state'] = json_data['tables'][j]['header']['flightphase']
    
    df = pd.concat([df, temp_df])

/Users/Navid/opt/anaconda3/envs/exjobb/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [200]:
df.head()

df = df[df.state == 'cruise']
df_1 = df.drop(['state', 'THRUST', 'DRAG'], axis=1)

In [201]:
df_1

,ALTITUDE,DISA,FUELFLOW,MACH,MASS
0,0.0,-30.0,0.231023,0.345534,39000.000
1,0.0,-30.0,0.242427,0.351389,44000.000
2,0.0,-30.0,0.257452,0.361062,49000.000
3,0.0,-30.0,0.273658,0.370924,54000.000
4,0.0,-30.0,0.290586,0.380586,59000.000
...,...,...,...,...,...
73281,39000.0,30.0,0.289282,0.740000,53380.024
73282,39800.0,30.0,0.234343,0.740000,39000.000
73283,39800.0,30.0,0.249693,0.740000,44000.000
73284,39800.0,30.0,0.269108,0.740000,49000.000


In [254]:
X = df_1.drop(['FUELFLOW'], axis=1)
y = df_1.FUELFLOW

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [255]:
test = X_test.iloc[0]
test

ALTITUDE     9000.00
DISA           30.00
MACH            0.46
MASS        44000.00
Name: 37153, dtype: float64

In [256]:
X_train['FUELFLOW'] = y_train
X_train = X_train.append(X_test)

X_train.tail()

/Users/Navid/opt/anaconda3/envs/exjobb/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ALTITUDE,DISA,FUELFLOW,MACH,MASS
46333,22000.0,30.0,NaN,0.54,79000.0
39202,2000.0,30.0,NaN,0.48,49000.0
17310,25000.0,-10.0,NaN,0.81,79000.0
32778,13000.0,10.0,NaN,0.42,74000.0
60695,29000.0,-10.0,NaN,0.66,80000.0


In [266]:
from scipy.interpolate import NearestNDInterpolator

myInterpolator = NearestNDInterpolator(X_train.drop(['FUELFLOW'],axis=1), X_train.FUELFLOW)
#X_train.interpolate(axis=1) 

y = myInterpolator()

In [ ]:
X_train[X_train.index.isin(X_test.index)]

In [259]:
len(y_test)

15770

In [189]:
pd.DataFrame([y_test,y_inter], columns=['y_test', 'y_inter'])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [179]:
mean_squared_error(y_test,y_inter)

ValueError: Found input variables with inconsistent numbers of samples: [15770, 16767]